In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sympy import symbols, Eq, solve
from datetime import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

from scipy.optimize import minimize, Bounds, show_options, basinhopping, brute, differential_evolution, dual_annealing
import math


# Basic Setup

In [ ]:
%%capture
%run -i basic_dataframe_setup.ipynb

In [ ]:
from glide_path_functions.manual_glide_paths import sigmoid_glide_path, linear_glide_path, other_glide_path, other2_glide_path, constant_leverage, normalized_sigmoid, book_path
from dataframe_utils import calculate_growth_with_glide_path, calculate_all_amounts_with_glide_paths, visualize, calculate_amount_with_glide_path_opt
from graph_utils import box_plot_comparison_of_glide_paths, line_graph_of_each_glide_path
from utils import calc_quartile_avg

## Graph of the Manual Glide Path Functions

In [ ]:
glide_path_functions = [
    ("Sigmoid Glide Path", sigmoid_glide_path),
    ("Linear Glide Path", linear_glide_path),
    ("Other Glide Path", other_glide_path),
    ("Other Glide Path 2", other2_glide_path),
    ("Constant Leverage", constant_leverage),
    ("sigmoid_glide_path_test", normalized_sigmoid),
    ("Book Path", book_path)
]

line_graph_of_each_glide_path(glide_path_functions)

In [ ]:
list_of_glide_path_func = [book_path, sigmoid_glide_path, other_glide_path, other2_glide_path, constant_leverage]
months_list, leverage_list = calculate_all_amounts_with_glide_paths(df, list_of_glide_path_func)

In [ ]:
box_plot_comparison_of_glide_paths(leverage_list, list_of_glide_path_func)

# Optimizing Glide Paths

We see above that we can habe multiple diferent possible glide paths that can each produce different results for the box plots. We are unsure of the exact shape of the 'ideal' glide path but that is the goal for the rest of this notebook.

In [ ]:
from glide_path_functions.optimized_glide_paths import sigmoid, sigmoid_default, piecewise, piecewise_default, polynomial, normalized_sigmoid

In [ ]:
def optimize(list_of_var, func):
    def helper (amount):
        return func(amount, list_of_var)
    
    leverage_list_opt = calculate_amount_with_glide_path_opt(df, helper)
    avg = calc_quartile_avg(leverage_list_opt)
    
    result = -1. * avg
    return result

def sigmoid_optimize(list_of_var):
    return optimize(list_of_var, normalized_sigmoid)

def piecewise_optimize(list_of_var):
    return optimize(list_of_var, piecewise)

def polynomial_optimize(list_of_var):
    return optimize(list_of_var, polynomial)

guess_sigmoid = [7.52701453e+00, 2.08417665e-06, 6.22858109e+06, 6.72461394e-01]
guess_n_sigmoid = [6, 2, 5, 0.5]
guess_piecewise = [-3, 8 , -6, 8, -3, 10, -1, 20, 2, 4, 8]


bound_sigmoid = [
    (0.0, 9),
    (2e-07, 1e-05),
    (-5e+06, 10e+06),
    (0, 1.2)
]
bound_n_sigmoid = [
    (0.0, 9),
    (0.5, 10),
    (-1, 10),
    (0, 2)
]
bound_piecewise =[
    (-10, 10),
    (0, 50),
    (-10, 10),
    (0, 50),
    (-10, 10),
    (0, 50),
    (-10, 10),
    (0, 50),
    (0, 10),
    (0, 20),
    (0, 30),
]
bound_polynomial = [
    (-10, 10),
    (-10, 10),
    (-10, 10),
    (-10, 10),
    (-10, 10),
    (-5, 5),
]

In [ ]:
# Check if initial guess is within bounds
# if not all(lower <= x <= upper for x, (lower, upper) in zip(guess_piecewise, bound_piecewise)):
#     print ("Initial guess is not within the specified bounds")
# show_options(solver='minimize', method='COBYLA', disp=True)

In [ ]:
#accept is not per step, it is per iteration (per local minima)
def accept(**kwargs):
    xnew = kwargs['x_new']
    print("test")
    for xi, (low, high) in zip(xnew, bound_piecewise):
        if not (low <= xi <= high):
            return False
    return True

result = minimize(piecewise_optimize, guess_piecewise, method='L-BFGS-B', bounds=bound_piecewise, options={'gtol': 1e-6})
#result = differential_evolution(sigmoid_optimize, bound_sigmoid, disp=True) #need to try dual annuel

#piecewise_minimizer_kwargs = { "method": "Nelder-Mead","bounds":bound_piecewise }
#basinhopping(piecewise_optimize, guess_piecewise, T=2, accept_test=accept, minimizer_kwargs=piecewise_minimizer_kwargs, disp=True, niter_success=1)

In [ ]:
print(result.x)
print(result.fun)
print(result.success)
print(result.message)

temp_opt = [x for x in result.x]
print(len(temp_opt))

## Guesses

1. [7.61645333e+00 1.42234210e-06 8.68306690e+06 6.21570943e-01] => -7.236472384747586

    This is for sigmoid_opt of default_start_amount = 10000, default_contribution_amount = 5000, default_month_interval = 12 * 2.5, default_difference = .000

2. [-2.97517529e-06  8.04051349e+00 -1.01686877e-06  8.72164523e+00 -7.45196808e-07  9.53897600e+00 -4.31036545e-06  8.00850297e+00] => -7.087004155096324

    This is for piecewise of default_start_amount = 10000, default_contribution_amount = 5000, default_month_interval = 12 * 2.5, default_difference = .000 with set x values

3. [-2.17423293e-07  8.38021564e+00 -4.05258433e-06  6.39217193e+00 -9.82229827e-06  9.32770644e+00 -1.10854224e-07  2.22762317e+00 4.96697812e+06  2.43985089e+06  1.83686376e+06] => -7.111982124971696

    This is for piecewise of default_start_amount = 10000, default_contribution_amount = 5000, default_month_interval = 12 * 2.5, default_difference = .000 with varying x values.

4. [-5.25354450e-08  8.36252777e+00 -1.59055917e-06  5.37105628e+00
 -3.33447881e-06  3.21115594e+00 -2.16527293e-07  3.54469172e+00
  4.48643019e+06  2.93376744e+06  2.47906832e+06  6.80476220e-01
  7.94358320e+00]
-7.16883075317587

5. [-1.52220080e-07  8.46270441e+00 -3.10300019e-06  6.63992385e+00
 -2.47083369e-06  3.19308924e+00 -8.11634160e-07  9.79006055e+00
  3.53446409e+06  2.33426388e+06  3.39381660e+06  8.26682453e-01
  7.83147076e+00]
-7.199014790128642

6. [7.79498742e+00 6.78299963e-06 7.59723102e+06 1.09826310e+00]
-7.315850902854188

7. [-3.58192185e-07  7.50307722e+00 -3.90285116e-06  3.33148209e+00
 -6.86964144e-06  6.22891024e-01 -6.07844579e-08  2.68487418e+00
  4.20075564e+06  4.68437616e+06  8.40069262e+05]
-7.173083995208977 

    This is unbounded piecewise, it goes into negative leverage which is a problem. Current problem is that if we (min max) the bounds of the leverage function, i am not sure if this correctly utilizing the optimize func because any paramters that exceed the bounds of lev func will result in the same value.


8. [-0.4716598   6.87965871 -2.85194484  7.62151847  0.63390778  4.38047322]
-7.420124470290059

    This is polynomial that achieved a negative value somehow


9. [8.99967086e+00 3.03509188e-06 5.02974074e+06 8.89664075e-01]
-7.061722612608467

    This is dual annualeing for sigmoid.
    
## Normalzied sigmoid
10. [7.60560019 2.4686383  5.54660909 0.772898  ] = -7.047447257735126

11. [8.59389735 2.83805718 5.01756339 0.91160879] = -7.071055942899254 DD -.001

12. [2.30399719 0.82899821 1.         0.        ] = -6.40649749289058 DD -.003

13. [1.57473466 2.33058679 3.14965699 0.34092716] = -6.505114042523816 DD -.002

14. [1.59613352 4.05636477 6.07167297 0.21810374] = -6.510032975958444 DD -.002 Nedler-Mead

15. [1.49806639 2.43540902 3.09735225 0.37342204] = -6.505196146714003 DD -.002 regular BFGS

In [ ]:
temp_opt = [8.59389735, 2.83805718, 5.01756339, 0.91160879]

In [ ]:
amounts = np.linspace(0, 15000000, 1000)  # From 0 to 8,000,000
glide_path_values = [normalized_sigmoid(amount, temp_opt) for amount in amounts]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(amounts, glide_path_values, label='Glide Path')

scale_factor = 1e6
ticks = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_factor))
plt.gca().xaxis.set_major_formatter(ticks)
plt.title('Glide Path Function Graph')
plt.xlabel('Amount')
plt.ylabel('Glide Path Value')
plt.grid(True)
plt.show()

## Visualize growth graph for each 30-year period


In [ ]:
single_list = [lambda x: normalized_sigmoid(x, temp_opt)]
visualize_df = visualize(df, single_list)

In [ ]:
# Plot each column with an offset
plt.figure(figsize=(10, 6))
offset_value = 1  # Define the offset value
for column in visualize_df.columns:
    plt.plot(visualize_df.index, visualize_df[column], label=column)

plt.xlabel('Index')
plt.ylabel('Value')
plt.yscale('log')
plt.title('Glide Path Proof')
plt.legend()
plt.show()

# Collecting data on multiple glide paths

In [ ]:
glide_path_functions = [
    ("Piecewise Default", piecewise_default),
    ("Sigmoid Glide Path", sigmoid_glide_path),
    ("Sigmoid Def.", sigmoid_default),
    ("Lambda", lambda x: normalized_sigmoid(x, temp_opt)),
]

line_graph_of_each_glide_path(glide_path_functions)

In [ ]:
list_of_glide_path_func = [book_path, other2_glide_path, lambda x: normalized_sigmoid(x, temp_opt), piecewise_default, sigmoid_glide_path, constant_leverage]
months_list, leverage_list = calculate_all_amounts_with_glide_paths(df, list_of_glide_path_func)
box_plot_comparison_of_glide_paths(leverage_list, list_of_glide_path_func)

In [ ]:
# s = pd.Series(calculate_amount_with_glide_path_opt(df, lambda x: sigmoid(x, temp_opt)))
# s.describe()

In [ ]:
# series_list = [pd.Series(inner_list) for inner_list in leverage_list]
# for list in series_list:
#     print(list.describe())

# Visulaize distrubution of months across glidepath functions

In [ ]:
series_months_list = [pd.Series(inner_list) for inner_list in months_list]
# for list in series_months_list:
#     print(list.describe())

In [ ]:
# plt.figure(figsize=(8, 12))
# plt.boxplot([series_months_list], positions=[1], showfliers=False)
# percentile_month = np.percentile(series_months_list, 90)
# plt.hlines(percentile_month, 0.9, 1.1, colors='blue', linestyles='dashed')
# plt.xticks([1], ['Months'])
# plt.title('Month Distribution')
# plt.ylabel('# of months')
# plt.show()


# Plotting side-by-side boxplots
plt.figure(figsize=(8, 12))
plt.boxplot(series_months_list, positions=[i for i in range(1,len(series_months_list) + 1)], showfliers=False)

# Calculate the xth percentile for each dataset
xth_perc = 98
for i, series in enumerate(series_months_list):
    percentile = np.percentile(series, xth_perc)
    plt.hlines(percentile, i + 0.9, i + 1.1, linestyles='dashed')

func_names = []
func_names.append
plt.xticks([i for i in range(1,len(series_months_list) + 1)], [func.__name__ for func in list_of_glide_path_func])
plt.xticks(rotation=45)
plt.title('Distrubtion of months till launch')
plt.ylabel('# of months till passed half mark')
plt.show()